# Content-Based Movie Recommender

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
warnings.filterwarnings("ignore")

X_features = pd.read_csv("../date/processed/X_features_only.csv")
titles = pd.read_csv("../date/processed/movie_titles.csv")
print(titles.head())
df_movies = pd.read_csv('../date/letterboxd_top_movies.csv')

In [ ]:
X_features = X_features.fillna(0)
similarity_matrix = cosine_similarity(X_features)

rint("\nDone!")
print(f"Similarity matrix shape: {similarity_matrix.shape}")
print(f"   → {similarity_matrix.shape[0]} movies × {similarity_matrix.shape[1]} movies")

In [ ]:
similarity_df = pd.DataFrame(
    similarity_matrix,
    index=titles["title"],
    columns=titles["title"]
)

print("\nExample: similarity of the first movie with the others:")
first_title = titles["title"].iloc[0]
print(f"\nMovie: {first_title}")

first_similarities = similarity_df.loc[first_title].sort_values(ascending=False).head(5)
print("\nTop 5 most similar movies (feature-based):")
print(first_similarities)
